In [ ]:
import os, random, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

In [ ]:
# 1) Load again to keep cells independent
(num_classes, base_input_shape) = (10, (32,32,3))
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.flatten()
y_test  = y_test.flatten()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=SEED, stratify=y_train
)

In [ ]:
# EfficientNetB0 expects 224x224 + specific preprocessing
IMG_SIZE = 224
preprocess = keras.applications.efficientnet.preprocess_input

In [ ]:
def make_ds(x, y, train=False, batch_size=64):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    if train:
        ds = ds.shuffle(10000, seed=SEED)
    # Resize and preprocess
    ds = ds.map(lambda a, b: (tf.image.resize(a, (IMG_SIZE, IMG_SIZE)), b),
                num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(lambda a, b: (preprocess(a), b), num_parallel_calls=tf.data.AUTOTUNE)
    if train:
        # Light augmentation on the fly
        aug = keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.05),
            layers.RandomZoom(0.1),
        ])
        ds = ds.map(lambda a, b: (aug(a, training=True), b), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
train_ds = make_ds(x_train, y_train, train=True, batch_size=128)
val_ds   = make_ds(x_val,   y_val,   train=False, batch_size=128)
test_ds  = make_ds(x_test,  y_test,  train=False, batch_size=128)

In [ ]:
# 2) Build transfer learning model
base = keras.applications.EfficientNetB0(
    include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base.trainable = False  # freeze base for warmup

In [ ]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model_tl = keras.Model(inputs, outputs, name="efficientnetb0_cifar10")

model_tl.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model_tl.summary()

In [ ]:
# 3) Warmup training (head only)
warmup = model_tl.fit(
    train_ds, validation_data=val_ds,
    epochs=5, verbose=2
)

In [ ]:
# 4) Fine-tune: unfreeze top of the base model
for layer in base.layers[-50:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

model_tl.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
early = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy")
fine = model_tl.fit(
    train_ds, validation_data=val_ds,
    epochs=20, callbacks=[early], verbose=2
)

In [ ]:
# 5) Evaluate + Metrics
probs = model_tl.predict(test_ds, verbose=0)
y_pred = probs.argmax(axis=1)

In [ ]:
# Bring true labels aligned with batched dataset order
y_true_list = []
for _, yb in test_ds:
    y_true_list.append(yb.numpy())
y_true = np.concatenate(y_true_list, axis=0)

acc = (y_pred == y_true).mean()
print(f"[EfficientNetB0 TL] Test Accuracy: {acc:.4f}")

print("\nClassification report (per-class precision/recall/F1):")
print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)

prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"Macro Precision: {prec:.4f} | Macro Recall: {rec:.4f} | Macro F1: {f1:.4f}")
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='micro')
print(f"Micro Precision: {prec:.4f} | Micro Recall: {rec:.4f} | Micro F1: {f1:.4f}")

In [ ]:
# ROC-AUC (one-vs-rest)
y_true_oh = keras.utils.to_categorical(y_true, num_classes)
try:
    auc_ovr = roc_auc_score(y_true_oh, probs, average="macro", multi_class="ovr")
    auc_ovo = roc_auc_score(y_true_oh, probs, average="macro", multi_class="ovo")
    print(f"ROC-AUC (macro, OVR): {auc_ovr:.4f} | ROC-AUC (macro, OVO): {auc_ovo:.4f}")
except Exception as e:
    print("ROC-AUC could not be computed:", e)